In [6]:
from diffusers import BitsAndBytesConfig, SD3Transformer2DModel
from diffusers import StableDiffusion3Pipeline
from transformers import MarianMTModel, MarianTokenizer
import torch
import re

In [2]:
model_id = "stabilityai/stable-diffusion-3.5-large"

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_nf4 = SD3Transformer2DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16
)

pipeline = StableDiffusion3Pipeline.from_pretrained(
    model_id, 
    transformer=model_nf4,
    torch_dtype=torch.bfloat16
)
pipeline.enable_model_cpu_offload()

Loading pipeline components...: 100%|██████████| 9/9 [00:03<00:00,  2.37it/s]


In [8]:
model_name = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [33]:
# 3. 한글 포함 여부 판별 함수
def contains_korean(text):
    return bool(re.search(r"[가-힣]", text))

In [34]:
# 4. 번역 함수
def translate_ko_to_en(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

In [51]:
prompt = "Trumph is striking Obama on the head with a baseball bat. Trumph has a menacing smile on her face, while Obama shows an expression of intense pain"
negative_prompt = "blurry, low quality, low resolution"

In [52]:
# 6. 한글이 포함되어 있으면 영어로 번역
if contains_korean(prompt):
    prompt = translate_ko_to_en(prompt)

In [53]:
print(prompt)

Trumph is striking Obama on the head with a baseball bat. Trumph has a menacing smile on her face, while Obama shows an expression of intense pain


In [54]:
image = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=100,
    guidance_scale=7.5,
    height=512,
    width=512,
    num_images_per_prompt=1,
    max_sequence_length=256,
).images[0]

100%|██████████| 100/100 [05:19<00:00,  3.19s/it]


In [55]:
image.save("output13.png")